In [193]:
from pprint import pprint 

# !pip install pymongo
import pymongo
myclient = pymongo.MongoClient("mongodb://localhost:27017/")

mydb = myclient["dd"]

items = mydb["items"]

### 1. Створіть декілька товарів з різним набором властивостей Phone/TV/Smart Watch/ .... 

Наприклад:
{
	"category" : "Phone",
	"model" : "iPhone 6",
	"producer" : "Apple",
	"price" : 600
}


In [194]:
dataset = [{
  "category": "Phone",
  "model": "iPhone 6",
  "producer": "Apple",
  "price": 600,
  "newField": "usual product"
},
{
  "category": "Laptop",
  "model": "MacBook Pro",
  "producer": "Apple",
  "price": 2400,
  "newField": "expensive product"
},
{
  "category": "Washing machine",
  "model": "LX-57348",
  "producer": "LG",
  "price": 750,
  "newField": "usual product"
},
{
  "category": "TV",
  "model": "SM-LED-FULL-PLUS",
  "producer": "Samsung",
  "price": 4200,
  "newField": "expensive product"
}]

items.insert_many(dataset)

InsertManyResult([ObjectId('65e78056217a595928553047'), ObjectId('65e78056217a595928553048'), ObjectId('65e78056217a595928553049'), ObjectId('65e78056217a59592855304a')], acknowledged=True)

### 2. Напишіть запит, який виводіть усі товари (відображення у JSON)

In [195]:
for x in items.find():
    pprint(x)

{'_id': ObjectId('65e75653217a595928552f00'),
 'category': 'Phone',
 'model': 'iPhone 6',
 'newField': 'expensive product',
 'price': 1322.0,
 'producer': 'Apple'}
{'_id': ObjectId('65e75653217a595928552f01'),
 'category': 'Laptop',
 'model': 'MacBook Pro',
 'newField': 'expensive product',
 'price': 3104.0000000000005,
 'producer': 'Apple'}
{'_id': ObjectId('65e75653217a595928552f02'),
 'category': 'Washing machine',
 'model': 'LX-57348',
 'newField': 'expensive product',
 'price': 1275.0,
 'producer': 'LG'}
{'_id': ObjectId('65e75653217a595928552f03'),
 'category': 'TV',
 'model': 'SM-LED-FULL-PLUS',
 'newField': 'expensive product',
 'price': 4620.0,
 'producer': 'Samsung'}
{'_id': ObjectId('65e78056217a595928553047'),
 'category': 'Phone',
 'model': 'iPhone 6',
 'newField': 'usual product',
 'price': 600,
 'producer': 'Apple'}
{'_id': ObjectId('65e78056217a595928553048'),
 'category': 'Laptop',
 'model': 'MacBook Pro',
 'newField': 'expensive product',
 'price': 2400,
 'producer': 

### 3. Підрахуйте скільки товарів у певної категорії

In [196]:
query = [
  {
    "$match": {
      "category": "Phone",
    },
  },
  {
    "$count":
      "Phone amount",
  },
    ]

for x in items.aggregate(query):
    pprint(x)

{'Phone amount': 2}


### 4. Підрахуйте скільки є різних категорій товарів

In [197]:
query = [
  {
    "$group":
      {
        "_id": "$category",
      },
  },
  {
    "$count":
      "Unique Categories",
  },
]

for x in items.aggregate(query):
    pprint(x)

{'Unique Categories': 4}


### 5. Виведіть список всіх виробників товарів без повторів

In [198]:
query = [
  {
    "$group": {
      "_id": "$producer",
    },
  },
]

for x in items.aggregate(query):
    pprint(x)

{'_id': 'Apple'}
{'_id': 'LG'}
{'_id': 'Samsung'}


### 6. Напишіть запити, які вибирають товари за різними критеріям і їх сукупності: 

#### - категорія та ціна (в проміжку) - конструкція $and, 



#### - модель чи одна чи інша - конструкція $or,


#### - виробники з переліку - конструкція $in

In [199]:
query_a = {"$and": [{"price": {"$lt": 1000}}, {"price": {"$gte": 500}}]}

print('6.A result')
for x in items.find(query_a):
    pprint(x)

'6.A result'
{'_id': ObjectId('65e78056217a595928553047'),
 'category': 'Phone',
 'model': 'iPhone 6',
 'newField': 'usual product',
 'price': 600,
 'producer': 'Apple'}
{'_id': ObjectId('65e78056217a595928553049'),
 'category': 'Washing machine',
 'model': 'LX-57348',
 'newField': 'usual product',
 'price': 750,
 'producer': 'LG'}


In [200]:
query_b = {"$or": [{ "model":'iPhone 6'}, {"model": "MacBook Pro"}]}

print('6.B result')
for x in items.find(query_b):
    pprint(x)

'6.B result'
{'_id': ObjectId('65e75653217a595928552f00'),
 'category': 'Phone',
 'model': 'iPhone 6',
 'newField': 'expensive product',
 'price': 1322.0,
 'producer': 'Apple'}
{'_id': ObjectId('65e75653217a595928552f01'),
 'category': 'Laptop',
 'model': 'MacBook Pro',
 'newField': 'expensive product',
 'price': 3104.0000000000005,
 'producer': 'Apple'}
{'_id': ObjectId('65e78056217a595928553047'),
 'category': 'Phone',
 'model': 'iPhone 6',
 'newField': 'usual product',
 'price': 600,
 'producer': 'Apple'}
{'_id': ObjectId('65e78056217a595928553048'),
 'category': 'Laptop',
 'model': 'MacBook Pro',
 'newField': 'expensive product',
 'price': 2400,
 'producer': 'Apple'}


In [201]:
query_c = {"producer": {"$in": ['Apple', 'Samsung']}}

print('6.C result')
for x in items.find(query_c):
    pprint(x)

'6.C result'
{'_id': ObjectId('65e75653217a595928552f00'),
 'category': 'Phone',
 'model': 'iPhone 6',
 'newField': 'expensive product',
 'price': 1322.0,
 'producer': 'Apple'}
{'_id': ObjectId('65e75653217a595928552f01'),
 'category': 'Laptop',
 'model': 'MacBook Pro',
 'newField': 'expensive product',
 'price': 3104.0000000000005,
 'producer': 'Apple'}
{'_id': ObjectId('65e75653217a595928552f03'),
 'category': 'TV',
 'model': 'SM-LED-FULL-PLUS',
 'newField': 'expensive product',
 'price': 4620.0,
 'producer': 'Samsung'}
{'_id': ObjectId('65e78056217a595928553047'),
 'category': 'Phone',
 'model': 'iPhone 6',
 'newField': 'usual product',
 'price': 600,
 'producer': 'Apple'}
{'_id': ObjectId('65e78056217a595928553048'),
 'category': 'Laptop',
 'model': 'MacBook Pro',
 'newField': 'expensive product',
 'price': 2400,
 'producer': 'Apple'}
{'_id': ObjectId('65e78056217a59592855304a'),
 'category': 'TV',
 'model': 'SM-LED-FULL-PLUS',
 'newField': 'expensive product',
 'price': 4200,
 'pr

### 7. Оновить певні товари, змінивши існуючі значення і додайте нові властивості (характеристики) усім товарам за певним критерієм

In [202]:
query_update_existing_field = [
  {
    "$set": {
      "price": {
        "$cond": {
          "if": {
            "$gte": ["$price", 1000],
          },
          "then": {"$multiply":["$price", 1.1]},
          "else": {"$multiply":["$price", 1.7]},
        },
      },
    },
  },
]

query_update_add_new_field = [
  {
    "$set": {
      "newField": {
        "$cond": {
          "if": {
            "$gte": ["$price", 1000],
          },
          "then": "expensive product",
          "else": "usual product",
        },
      },
    },
  },
]

# Change value for price
items.update_many({}, query_update_existing_field)

# Add new value base on price query
items.update_many({}, query_update_add_new_field)

# OUTPUT new dataset
for x in items.find():
    print(x)

{'_id': ObjectId('65e75653217a595928552f00'),
 'category': 'Phone',
 'model': 'iPhone 6',
 'newField': 'expensive product',
 'price': 1454.2,
 'producer': 'Apple'}
{'_id': ObjectId('65e75653217a595928552f01'),
 'category': 'Laptop',
 'model': 'MacBook Pro',
 'newField': 'expensive product',
 'price': 3414.400000000001,
 'producer': 'Apple'}
{'_id': ObjectId('65e75653217a595928552f02'),
 'category': 'Washing machine',
 'model': 'LX-57348',
 'newField': 'expensive product',
 'price': 1402.5,
 'producer': 'LG'}
{'_id': ObjectId('65e75653217a595928552f03'),
 'category': 'TV',
 'model': 'SM-LED-FULL-PLUS',
 'newField': 'expensive product',
 'price': 5082.0,
 'producer': 'Samsung'}
{'_id': ObjectId('65e78056217a595928553047'),
 'category': 'Phone',
 'model': 'iPhone 6',
 'newField': 'expensive product',
 'price': 1020.0,
 'producer': 'Apple'}
{'_id': ObjectId('65e78056217a595928553048'),
 'category': 'Laptop',
 'model': 'MacBook Pro',
 'newField': 'expensive product',
 'price': 2640.0,
 'pro

### 8. Знайдіть товари у яких є (присутнє поле) певні властивості
### 9. Для знайдених товарів збільшіть їх вартість на певну суму

In [203]:
query_8 = {
        "producer": "Apple",
      }

for x in items.find(query_8):
    pprint(x)


      
query_9 = [{
    "$set": {
      "price": {"$add":["$price", 200]},
      },
    }]

items.update_many(query_8, query_9)

{'_id': ObjectId('65e75653217a595928552f00'),
 'category': 'Phone',
 'model': 'iPhone 6',
 'newField': 'expensive product',
 'price': 1454.2,
 'producer': 'Apple'}
{'_id': ObjectId('65e75653217a595928552f01'),
 'category': 'Laptop',
 'model': 'MacBook Pro',
 'newField': 'expensive product',
 'price': 3414.400000000001,
 'producer': 'Apple'}
{'_id': ObjectId('65e78056217a595928553047'),
 'category': 'Phone',
 'model': 'iPhone 6',
 'newField': 'expensive product',
 'price': 1020.0,
 'producer': 'Apple'}
{'_id': ObjectId('65e78056217a595928553048'),
 'category': 'Laptop',
 'model': 'MacBook Pro',
 'newField': 'expensive product',
 'price': 2640.0,
 'producer': 'Apple'}


UpdateResult({'n': 4, 'nModified': 4, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [204]:
# OUTPUT new dataset
for x in items.find():
    print(x)

{'_id': ObjectId('65e75653217a595928552f00'),
 'category': 'Phone',
 'model': 'iPhone 6',
 'newField': 'expensive product',
 'price': 1654.2,
 'producer': 'Apple'}
{'_id': ObjectId('65e75653217a595928552f01'),
 'category': 'Laptop',
 'model': 'MacBook Pro',
 'newField': 'expensive product',
 'price': 3614.400000000001,
 'producer': 'Apple'}
{'_id': ObjectId('65e75653217a595928552f02'),
 'category': 'Washing machine',
 'model': 'LX-57348',
 'newField': 'expensive product',
 'price': 1402.5,
 'producer': 'LG'}
{'_id': ObjectId('65e75653217a595928552f03'),
 'category': 'TV',
 'model': 'SM-LED-FULL-PLUS',
 'newField': 'expensive product',
 'price': 5082.0,
 'producer': 'Samsung'}
{'_id': ObjectId('65e78056217a595928553047'),
 'category': 'Phone',
 'model': 'iPhone 6',
 'newField': 'expensive product',
 'price': 1220.0,
 'producer': 'Apple'}
{'_id': ObjectId('65e78056217a595928553048'),
 'category': 'Laptop',
 'model': 'MacBook Pro',
 'newField': 'expensive product',
 'price': 2840.0,
 'pro

### Товари ви додаєте в замовлення - orders, яке містити вартість, ім'я замовника, і адресу доставки.
#### Товари (items) в замовленні (order) повинні бути представлені у вигляді references, а замовник (customer) у вигляді embed
##### Приклад моделювання: https://docs.mongodb.com/manual/tutorial/model-referenced-one-to-many-relationships-between-documents/ 


In [205]:
orders = mydb["orders"]
orders.delete_many({})

DeleteResult({'n': 3, 'ok': 1.0}, acknowledged=True)

### 1. Створіть кілька замовлень з різними наборами товарів, але так щоб один з товарів був у декількох замовленнях

In [206]:
dataset = [
    {
      "order_number": 201513,
      "date": "2015-04-14",
      "total_sum": 1923.4,
      "customer": {
        "name": "Andrii",
        "surname": "Rodinov",
        "phones": [
          9876543,
          1234567
        ],
        "address": "PTI, Peremohy 37, Kyiv, UA"
      },
      "payment": {
        "card_owner": "Andrii Rodionov",
        "cardId": 12345678
      },
      "items_id": [
        "65e75653217a595928552f00",
        "65e75653217a595928552f01"
      ]
    },
    {
      "order_number": 201513,
      "date": "2023-04-14",
      "total_sum": 1923.4,
      "customer": {
        "name": "Nikita",
        "surname": "Ochinskiy",
        "phones": [
          312312412,
          4123213
        ],
        "address": "PTI, Peremohy 37, Odesa, UA"
      },
      "payment": {
        "card_owner": "Nikita Ochinskiy",
        "cardId": {
          "$numberLong": "1234567812398723"
        }
      },
      "items_id": [
        "65e75653217a595928552f00",
        "65e75653217a595928552f02"
      ]
    },
    {
      "order_number": 4911134,
      "date": "2012-04-14",
      "total_sum": 1923.4,
      "customer": {
        "name": "Loza",
        "surname": "Bandura",
        "phones": [
          {
            "$numberLong": "98765435334"
          },
          1234567434
        ],
        "address": "PTI, Peremohy 37, Kyiv, UA"
      },
      "payment": {
        "card_owner": "Loza Bandura",
        "cardId": {
          "$numberLong": "12345678123124"
        }
      },
      "items_id": [
        "65e75653217a595928552f02",
        "65e75653217a595928552f03"
      ]
    },
    {
      "order_number": {
        "$numberLong": "4911134412"
      },
      "date": "2012-09-14",
      "total_sum": 1923.4,
      "customer": {
        "name": "Koza",
        "surname": "Zandura",
        "phones": [
          {
            "$numberLong": "98765435334"
          },
          1234567434
        ],
        "address": "PTI, Asdas 37, Kyiv, UA"
      },
      "payment": {
        "card_owner": "Loza Bsas",
        "cardId": {
          "$numberLong": "12345678123124"
        }
      },
      "items_id": [
        "65e75653217a595928552f01",
        "65e75653217a595928552f03"
      ]
    }
  ]

orders.insert_many(dataset)

InsertManyResult([ObjectId('65e7806a217a59592855304b'), ObjectId('65e7806a217a59592855304c'), ObjectId('65e7806a217a59592855304d'), ObjectId('65e7806a217a59592855304e')], acknowledged=True)

### 2. Виведіть всі замовлення

In [207]:
for x in orders.find():
    pprint(x)

{'_id': ObjectId('65e7806a217a59592855304b'),
 'customer': {'address': 'PTI, Peremohy 37, Kyiv, UA',
              'name': 'Andrii',
              'phones': [9876543, 1234567],
              'surname': 'Rodinov'},
 'date': '2015-04-14',
 'items_id': ['65e75653217a595928552f00', '65e75653217a595928552f01'],
 'order_number': 201513,
 'payment': {'cardId': 12345678, 'card_owner': 'Andrii Rodionov'},
 'total_sum': 1923.4}
{'_id': ObjectId('65e7806a217a59592855304c'),
 'customer': {'address': 'PTI, Peremohy 37, Odesa, UA',
              'name': 'Nikita',
              'phones': [312312412, 4123213],
              'surname': 'Ochinskiy'},
 'date': '2023-04-14',
 'items_id': ['65e75653217a595928552f00', '65e75653217a595928552f02'],
 'order_number': 201513,
 'payment': {'cardId': {'$numberLong': '1234567812398723'},
             'card_owner': 'Nikita Ochinskiy'},
 'total_sum': 1923.4}
{'_id': ObjectId('65e7806a217a59592855304d'),
 'customer': {'address': 'PTI, Peremohy 37, Kyiv, UA',
         

### 3. Виведіть замовлення з вартістю більше певного значення

In [208]:
query =  {
    "total_sum": {
      "$gte": 1000.0
    }
  }

for x in orders.find(query):
    pprint(x)

{'_id': ObjectId('65e7806a217a59592855304b'),
 'customer': {'address': 'PTI, Peremohy 37, Kyiv, UA',
              'name': 'Andrii',
              'phones': [9876543, 1234567],
              'surname': 'Rodinov'},
 'date': '2015-04-14',
 'items_id': ['65e75653217a595928552f00', '65e75653217a595928552f01'],
 'order_number': 201513,
 'payment': {'cardId': 12345678, 'card_owner': 'Andrii Rodionov'},
 'total_sum': 1923.4}
{'_id': ObjectId('65e7806a217a59592855304c'),
 'customer': {'address': 'PTI, Peremohy 37, Odesa, UA',
              'name': 'Nikita',
              'phones': [312312412, 4123213],
              'surname': 'Ochinskiy'},
 'date': '2023-04-14',
 'items_id': ['65e75653217a595928552f00', '65e75653217a595928552f02'],
 'order_number': 201513,
 'payment': {'cardId': {'$numberLong': '1234567812398723'},
             'card_owner': 'Nikita Ochinskiy'},
 'total_sum': 1923.4}
{'_id': ObjectId('65e7806a217a59592855304d'),
 'customer': {'address': 'PTI, Peremohy 37, Kyiv, UA',
         

### 4. Знайдіть замовлення зроблені одним замовником,


In [209]:
query =  {
    "customer.surname": "Rodinov",
    "customer.name": "Andrii"
  }

for x in orders.find(query):
    pprint(x)

{'_id': ObjectId('65e7806a217a59592855304b'),
 'customer': {'address': 'PTI, Peremohy 37, Kyiv, UA',
              'name': 'Andrii',
              'phones': [9876543, 1234567],
              'surname': 'Rodinov'},
 'date': '2015-04-14',
 'items_id': ['65e75653217a595928552f00', '65e75653217a595928552f01'],
 'order_number': 201513,
 'payment': {'cardId': 12345678, 'card_owner': 'Andrii Rodionov'},
 'total_sum': 1923.4}


### 5. Знайдіть всі замовлення з певним товаром (товарами) (шукати можна по ObjectId),


In [210]:
query =  {
    "items_id": '65e75653217a595928552f03'
  }

for x in orders.find(query):
    pprint(x)

{'_id': ObjectId('65e7806a217a59592855304d'),
 'customer': {'address': 'PTI, Peremohy 37, Kyiv, UA',
              'name': 'Loza',
              'phones': [{'$numberLong': '98765435334'}, 1234567434],
              'surname': 'Bandura'},
 'date': '2012-04-14',
 'items_id': ['65e75653217a595928552f02', '65e75653217a595928552f03'],
 'order_number': 4911134,
 'payment': {'cardId': {'$numberLong': '12345678123124'},
             'card_owner': 'Loza Bandura'},
 'total_sum': 1923.4}
{'_id': ObjectId('65e7806a217a59592855304e'),
 'customer': {'address': 'PTI, Asdas 37, Kyiv, UA',
              'name': 'Koza',
              'phones': [{'$numberLong': '98765435334'}, 1234567434],
              'surname': 'Zandura'},
 'date': '2012-09-14',
 'items_id': ['65e75653217a595928552f01', '65e75653217a595928552f03'],
 'order_number': {'$numberLong': '4911134412'},
 'payment': {'cardId': {'$numberLong': '12345678123124'},
             'card_owner': 'Loza Bsas'},
 'total_sum': 1923.4}


### 6. Додайте в усі замовлення з певним товаром ще один товар і збільште існуючу вартість замовлення на деяке значення Х,


In [211]:
query_find_item = {
        "items_id": "65e75653217a595928552f03"
      }

query_add_new_item = [{
      "$set": {
        "items_id": {
          "$concatArrays": [
            "$items_id",
            [
              "65e75653217a595928552f00"
            ]
          ]
        }
      }
    }]

X = 900
query_change_total_sum = [{
      "$set": {
        "total_sum": {
          "$add": [
            "$total_sum", X
          ]
        }
      }
    }]

orders.update_many(query_find_item, query_add_new_item)
orders.update_many(query_find_item, query_change_total_sum)

UpdateResult({'n': 2, 'nModified': 2, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [212]:
for x in orders.find():
    pprint(x)

{'_id': ObjectId('65e7806a217a59592855304b'),
 'customer': {'address': 'PTI, Peremohy 37, Kyiv, UA',
              'name': 'Andrii',
              'phones': [9876543, 1234567],
              'surname': 'Rodinov'},
 'date': '2015-04-14',
 'items_id': ['65e75653217a595928552f00', '65e75653217a595928552f01'],
 'order_number': 201513,
 'payment': {'cardId': 12345678, 'card_owner': 'Andrii Rodionov'},
 'total_sum': 1923.4}
{'_id': ObjectId('65e7806a217a59592855304c'),
 'customer': {'address': 'PTI, Peremohy 37, Odesa, UA',
              'name': 'Nikita',
              'phones': [312312412, 4123213],
              'surname': 'Ochinskiy'},
 'date': '2023-04-14',
 'items_id': ['65e75653217a595928552f00', '65e75653217a595928552f02'],
 'order_number': 201513,
 'payment': {'cardId': {'$numberLong': '1234567812398723'},
             'card_owner': 'Nikita Ochinskiy'},
 'total_sum': 1923.4}
{'_id': ObjectId('65e7806a217a59592855304d'),
 'customer': {'address': 'PTI, Peremohy 37, Kyiv, UA',
         

### 7. Виведіть кількість товарів в певному замовленні,


In [213]:
query = [
    {
      "$match": {
        "order_number": 4911134
      }
    },
    {
      "$unwind": {
        "path": "$items_id"
      }
    },
    {
      "$count": "Amount of items in order"
    }
  ]

for x in orders.aggregate(query):
    pprint(x)

{'Amount of items in order': 3}


### 8. Виведіть тільки інформацію про кастомера і номери кредитної карт, для замовлень вартість яких перевищує певну суму,


In [214]:
query = [
    {
      "$match": 
      {
        "total_sum": {
          "$gte": 2000
        }
      }
    },
    {
      "$group": {
        "_id": {
          "customer": "$customer",
          "cart_id": "$payment.cardId"
        }
      }
    }
  ]

for x in orders.aggregate(query):
    pprint(x)

{'_id': {'cart_id': {'$numberLong': '12345678123124'},
         'customer': {'address': 'PTI, Peremohy 37, Kyiv, UA',
                      'name': 'Loza',
                      'phones': [{'$numberLong': '98765435334'}, 1234567434],
                      'surname': 'Bandura'}}}
{'_id': {'cart_id': {'$numberLong': '12345678123124'},
         'customer': {'address': 'PTI, Asdas 37, Kyiv, UA',
                      'name': 'Koza',
                      'phones': [{'$numberLong': '98765435334'}, 1234567434],
                      'surname': 'Zandura'}}}


### 9. Видаліть товар з замовлень, зроблених за певний період дат


In [215]:
query = {
    "date": {
      "$gte": "2012-09-14",
      "$lte": "2014-04-13"
    }
  }

orders.delete_many(query)

DeleteResult({'n': 1, 'ok': 1.0}, acknowledged=True)

In [216]:
for x in orders.find():
    pprint(x)

{'_id': ObjectId('65e7806a217a59592855304b'),
 'customer': {'address': 'PTI, Peremohy 37, Kyiv, UA',
              'name': 'Andrii',
              'phones': [9876543, 1234567],
              'surname': 'Rodinov'},
 'date': '2015-04-14',
 'items_id': ['65e75653217a595928552f00', '65e75653217a595928552f01'],
 'order_number': 201513,
 'payment': {'cardId': 12345678, 'card_owner': 'Andrii Rodionov'},
 'total_sum': 1923.4}
{'_id': ObjectId('65e7806a217a59592855304c'),
 'customer': {'address': 'PTI, Peremohy 37, Odesa, UA',
              'name': 'Nikita',
              'phones': [312312412, 4123213],
              'surname': 'Ochinskiy'},
 'date': '2023-04-14',
 'items_id': ['65e75653217a595928552f00', '65e75653217a595928552f02'],
 'order_number': 201513,
 'payment': {'cardId': {'$numberLong': '1234567812398723'},
             'card_owner': 'Nikita Ochinskiy'},
 'total_sum': 1923.4}
{'_id': ObjectId('65e7806a217a59592855304d'),
 'customer': {'address': 'PTI, Peremohy 37, Kyiv, UA',
         

### 10. Перейменуйте у всіх замовлення ім'я (прізвище) замовника

In [217]:
query = [
    {
      "$set": {
        "customer.name": "Misha",
        "customer.surname": "Mezinov",
        "payment.card_owner": "Misha Mezinov"
      }
    }
  ]

orders.update_many({}, query)

UpdateResult({'n': 3, 'nModified': 3, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [218]:
for x in orders.find():
    pprint(x)

{'_id': ObjectId('65e7806a217a59592855304b'),
 'customer': {'address': 'PTI, Peremohy 37, Kyiv, UA',
              'name': 'Misha',
              'phones': [9876543, 1234567],
              'surname': 'Mezinov'},
 'date': '2015-04-14',
 'items_id': ['65e75653217a595928552f00', '65e75653217a595928552f01'],
 'order_number': 201513,
 'payment': {'cardId': 12345678, 'card_owner': 'Misha Mezinov'},
 'total_sum': 1923.4}
{'_id': ObjectId('65e7806a217a59592855304c'),
 'customer': {'address': 'PTI, Peremohy 37, Odesa, UA',
              'name': 'Misha',
              'phones': [312312412, 4123213],
              'surname': 'Mezinov'},
 'date': '2023-04-14',
 'items_id': ['65e75653217a595928552f00', '65e75653217a595928552f02'],
 'order_number': 201513,
 'payment': {'cardId': {'$numberLong': '1234567812398723'},
             'card_owner': 'Misha Mezinov'},
 'total_sum': 1923.4}
{'_id': ObjectId('65e7806a217a59592855304d'),
 'customer': {'address': 'PTI, Peremohy 37, Kyiv, UA',
              'nam

### 11. Знайдіть замовлення зроблені одним замовником, і виведіть тільки інформацію про кастомерата товари у замовлені підставивши замість ObjectId("***") назви товарів та їх вартість
  
#### (аналог join-а між таблицями orders та items).

In [219]:
query = [
    {
      "$unwind": {
        "path": "$items_id"
      }
    },
    {
      "$set": {
        "items_id": {
          "$toObjectId": "$items_id"
        }
      }
    },
    {
      "$lookup": {
        "from": "items",
        "localField": "items_id",
        "foreignField": "_id",
        "as": "item_params"
      }
    },
    {
      "$unwind": {
        "path": "$item_params"
      }
    },
    {
      "$group": {
        "_id": {
          "customer_name": "$customer.name",
          "customer_surname": "$customer.surname",
          "product_category": "$item_params.category",
          "product_model": "$item_params.model",
          "product_price": "$item_params.price"
        }
      }
    }
]

for x in orders.aggregate(query):
    print('--------------------------------------------------------------------------')
    pprint(x)

'--------------------------------------------------------------------------'
{'_id': {'customer_name': 'Misha',
         'customer_surname': 'Mezinov',
         'product_category': 'Laptop',
         'product_model': 'MacBook Pro',
         'product_price': 3614.400000000001}}
'--------------------------------------------------------------------------'
{'_id': {'customer_name': 'Misha',
         'customer_surname': 'Mezinov',
         'product_category': 'TV',
         'product_model': 'SM-LED-FULL-PLUS',
         'product_price': 5082.0}}
'--------------------------------------------------------------------------'
{'_id': {'customer_name': 'Misha',
         'customer_surname': 'Mezinov',
         'product_category': 'Washing machine',
         'product_model': 'LX-57348',
         'product_price': 1402.5}}
'--------------------------------------------------------------------------'
{'_id': {'customer_name': 'Misha',
         'customer_surname': 'Mezinov',
         'product_category': 

In [221]:
# feedbacks.drop()
feedbacks = mydb.create_collection('feedbacks', capped=True, size=1000000, max=5)
feedbacks.delete_many({})

DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)

In [223]:
# I run 10 times 
for x in range(10):
    feedbacks.insert_one({'message': x})
    print(f'Added message {x} to list')

print('--------------------------------------------------------------------------')
print(f'And now I output whole collection, but it would has only 5 last documents')
print('--------------------------------------------------------------------------')

for x in feedbacks.find():
    print(x)

'Added message 0 to list'
'Added message 1 to list'
'Added message 2 to list'
'Added message 3 to list'
'Added message 4 to list'
'Added message 5 to list'
'Added message 6 to list'
'Added message 7 to list'
'Added message 8 to list'
'Added message 9 to list'
'--------------------------------------------------------------------------'
'And now I output whole collection, but it would has only 5 last documents'
'--------------------------------------------------------------------------'
{'_id': ObjectId('65e7809f217a59592855305e'), 'message': 5}
{'_id': ObjectId('65e7809f217a59592855305f'), 'message': 6}
{'_id': ObjectId('65e7809f217a595928553060'), 'message': 7}
{'_id': ObjectId('65e7809f217a595928553061'), 'message': 8}
{'_id': ObjectId('65e7809f217a595928553062'), 'message': 9}
